<a href="https://colab.research.google.com/github/ishaandindwar/my-projects/blob/main/number_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from sklearn.datasets import fetch_openml

mist = fetch_openml('mnist_784', version =1, as_frame = False,parser ='auto')

X_data = mist.data.astype(np.float32)
y_target = mist.target.astype(np.int64) # MNIST labels are typically integers

train_pixels_raw = X_data[:60000]
train_labels = y_target[:60000]
test_pixels_raw = X_data[60000:]
test_labels = y_target[60000:]


def deskew(img):
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    # Calculate skew based on central moments
    skew = m['mu11'] / m['mu02']
    M = np.float32([[1, skew, -0.5 * 28 * skew], [0, 1, 0]])
    img = cv2.warpAffine(img, M, (28, 28), flags=cv2.WARP_INVERSE_MAP | cv2.INTER_LINEAR)
    return img

def preprocess_batch(pixels):
    deskewed_batch = []
    for p in pixels:
        img = p.reshape(28, 28)
        img = deskew(img)
        deskewed_batch.append(img.flatten())
    return np.array(deskewed_batch, dtype=np.float32)

print("Preprocessing (Deskewing)...")
train_pixels_deskewed = preprocess_batch(train_pixels_raw) / 255.0
test_pixels_deskewed = preprocess_batch(test_pixels_raw) / 255.0

Preprocessing (Deskewing)...


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# --- 1. Reshape Data for CNN ---
# CNNs expect (batch_size, height, width, channels)
X_train_cnn = train_pixels_deskewed.reshape(-1, 28, 28, 1)
X_test_cnn = test_pixels_deskewed.reshape(-1, 28, 28, 1)

# --- 2. Build the CNN Architecture ---
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax') # 10 output classes for digits 0-9
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# --- 3. Train the Model ---
print("Training CNN... This may take a minute or two on CPU.")
model.fit(X_train_cnn, train_labels, epochs=5, batch_size=64, validation_split=0.1)

# --- 4. Accuracy Check ---
test_loss, test_acc = model.evaluate(X_test_cnn, test_labels, verbose=0)
print(f"\n✅ Training Complete. CNN Model Accuracy: {test_acc*100:.2f}%")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training CNN... This may take a minute or two on CPU.
Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 56s 64ms/step - accuracy: 0.8974 - loss: 0.3655 - val_accuracy: 0.9818 - val_loss: 0.0597
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 77s 58ms/step - accuracy: 0.9849 - loss: 0.0487 - val_accuracy: 0.9877 - val_loss: 0.0377
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 79s 55ms/step - accuracy: 0.9898 - loss: 0.0330 - val_accuracy: 0.9917 - val_loss: 0.0298
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 45s 54ms/step - accuracy: 0.9933 - loss: 0.0224 - val_accuracy: 0.9890 - val_loss: 0.0351
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 84s 56ms/step - accuracy: 0.9937 - loss: 0.0207 - val_accuracy: 0.9900 - val_loss: 0.0411

✅ Training Complete. CNN Model Accuracy: 99.17%


In [ ]:
from google.colab import output
from IPython.display import HTML, display
import base64
import tensorflow as tf # Import tensorflow to access the global model object

# This HTML/JS creates the drawing interface
canvas_html = """
<div style="border: 2px solid #444; padding: 20px; border-radius: 10px; background-color: #f9f9f9; width: 300px; text-align: center;">
    <h3 style="margin-top: 0; color: #333;">Draw a Digit (0-9)</h3>
    <canvas id="canvas" width="280" height="280" style="border: 2px solid #000; background-color: black; cursor: crosshair;"></canvas>
    <br><br>
    <button id="predict" style="padding: 10px 20px; background: #28a745; color: white; border: none; border-radius: 5px; cursor: pointer;">Predict</button>
    <button id="clear" style="padding: 10px 20px; background: #dc3545; color: white; border: none; border-radius: 5px; cursor: pointer;">Clear</button>
</div>

<script>
    var canvas = document.getElementById('canvas');
    var ctx = canvas.getContext('2d');
    ctx.strokeStyle = 'white';
    ctx.lineWidth = 20;
    ctx.lineCap = 'round';
    var drawing = false;

    canvas.onmousedown = (e) => { drawing = true; ctx.beginPath(); ctx.moveTo(e.offsetX, e.offsetY); };
    canvas.onmousemove = (e) => { if(drawing) { ctx.lineTo(e.offsetX, e.offsetY); ctx.stroke(); } };
    canvas.onmouseup = () => { drawing = false; };

    document.getElementById('clear').onclick = () => { ctx.clearRect(0, 0, canvas.width, canvas.height); };

    document.getElementById('predict').onclick = () => {
        var dataURL = canvas.toDataURL('image/png');
        google.colab.kernel.invokeFunction('notebook.PredictDigit', [dataURL], {});
    };
</script>
"""

def PredictDigit(data_url):
    # 1. Decode base64 image from JS
    header, encoded = data_url.split(",", 1)
    data = base64.b64decode(encoded)

    # 2. Convert to OpenCV format
    nparr = np.frombuffer(data, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 3. Resize to 28x28 (MNIST size)
    resized = cv2.resize(gray, (28, 28), interpolation=cv2.INTER_AREA)

    # 4. Apply same preprocessing as training
    deskewed = deskew(resized)
    # Reshape for CNN: (batch_size, height, width, channels)
    final_input = deskewed.reshape(1, 28, 28, 1).astype(np.float32) / 255.0

    # 5. Predict using the trained Keras model
    predictions = model.predict(final_input)
    prediction = np.argmax(predictions) # Get the digit with the highest probability

    # Output the result clearly
    print(f"Prediction: {prediction}")

# Register the callback and display the UI
output.register_callback('notebook.PredictDigit', PredictDigit)
display(HTML(canvas_html))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
Prediction: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Prediction: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prediction: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Prediction: 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Prediction: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Prediction: 2
